In [1]:
import numpy as np
import time
from data import datagenerator
from data import datahandler
from bokeh.models import Range1d, PrintfTickFormatter
from bokeh.plotting import figure, show, Session, output_server, cursession

In [2]:
#session = Session(load_from_config=False, root_url = "http://127.0.0.1:5006/")
output_server("barchart", url = "http://127.0.0.1:5006/")

Using saved session configuration for http://127.0.0.1:5006/
To override, pass 'load_from_config=False' to Session


In [3]:
class BarChart(object):
       
    #restituisce una lista di stringhe con nomi di colori di numero di elementi argomento, a partire da una lista
    #di stringhe con nomi di colori vista come ciclica
    def colorlist(self, colors, bar_interval_n):
        colorlist = [colors[i%len(colors)] for i in range(bar_interval_n)]
        return colorlist
    
    #restituisce un lista di tante liste quante le barre del bar chart. Ciascuna lista contiene un numero di elementi 
    #uguale agli intervalli del plot e con valore uguale all'ascissa sinistra della rispettiva barra
    def leftlists(self, barwidth, bar_interval_n, bar_n):
        leftlists = [[(j - barwidth / 2) for i in range(bar_interval_n)] for j in range(1, bar_n + 1)]
        return leftlists
    
    #restituisce un lista di tante liste quante le barre del bar chart. Ciascuna lista contiene un numero di elementi 
    #uguale agli intervalli del plot e con valore uguale all'ascissa destra della rispettiva barra
    def rightlists(self, barwidth, bar_interval_n, bar_n):
        rightlists = [[(j + barwidth / 2) for i in range(bar_interval_n)] for j in range(1, bar_n + 1)]
        return rightlists
    
    #restituisce un lista di tante liste quante le barre del bar chart. Ciascuna lista contiene un numero di elementi 
    #uguale agli intervalli del plot con valori uguale all'estremo inferiore di ciascun intervallo
    def bottomlists(self, bar_interval_matrix):
        bottomlists = [bar_interval_list[:-1] for bar_interval_list in bar_interval_matrix]
        return bottomlists
    
    #restituisce un lista di tante liste quante le barre del bar chart. Ciascuna lista contiene un numero di elementi 
    #uguale agli intervalli del plot con valori uguale all'estremo superiore di ciascun intervallo
    def toplists(self, bar_interval_matrix):
        toplists = [bar_interval_list[1:] for bar_interval_list in bar_interval_matrix]
        return toplists

In [4]:
#Inizializzazione delle variabili

barchart = BarChart()

mean = 10
std = 2
delay = 0.1

var_mean = 0
var_std = 0.4
var_delay = 0.1

barwidth = 0.1 #larghezza compresa tra 0.0 e 1.0 (barre attaccate)
bar_n = 5
bar_interval_n = 4
colors =["red", "orange", "yellow", "green", "aqua", "blue", "purple", "brown", "grey", "black"]

In [5]:
datalists = datagenerator.normal_matrix(bar_n, bar_interval_n, mean, std, delay)
cumsumdatalists = datahandler.cumsum_matrix(datalists)
bar_interval_lists = datahandler.zeroappend_matrix(cumsumdatalists)

In [6]:
toplists = barchart.toplists(bar_interval_lists)
bottomlists = barchart.bottomlists(bar_interval_lists)
leftlists = barchart.leftlists(barwidth, bar_interval_n, bar_n)
rightlists = barchart.rightlists(barwidth, bar_interval_n, bar_n)
colorlist = barchart.colorlist(colors, bar_interval_n)

In [7]:
p = figure(
    plot_width = 1000,
    plot_height = 600,
    x_range = Range1d(0.5, bar_n + 0.5),
    y_range = Range1d(- 0.05 * mean * bar_interval_n, (mean * bar_interval_n)*1.5),
)
p.xgrid.grid_line_color = None
p.xaxis.major_tick_line_color = None
p.xaxis.minor_tick_line_color = None
p.xaxis[0].formatter = PrintfTickFormatter(format="LOG %d")

In [8]:
for bar in range(bar_n):
    p.quad(
        top = toplists[bar],
        bottom = bottomlists[bar],
        left = leftlists[bar],
        right = rightlists[bar],
        line_color = colorlist,
        fill_color = colorlist,
        name = "bar"
    )

In [9]:
show(p)

ds = [p.select({"name":"bar"})[i].data_source for i in range(bar_n)]

while True:
    variancelists = datagenerator.normal_matrix(bar_n, bar_interval_n, var_mean, var_std, var_delay)
    
    datalists = np.maximum(0, datahandler.sum_list(datalists, variancelists))
    cumsumdatalists = datahandler.cumsum_matrix(datalists)
    bar_interval_lists = datahandler.zeroappend_matrix(cumsumdatalists)
    
    toplists = barchart.toplists(bar_interval_lists)
    bottomlists = barchart.bottomlists(bar_interval_lists)
    
    for i in range(bar_n):
        p.select({"name":"bar"})[i].data_source.data["top"] = toplists[i]
        p.select({"name":"bar"})[i].data_source.data["bottom"] = bottomlists[i]
    cursession().store_objects(*ds)
    #cursession().pubblish()
    
    time.sleep(0.5)

KeyboardInterrupt: 